## Unbalanced power flow calculation with pandapower

We will create a simple three bus system to demonstrate how to run unbalanced power flow calculations with pandapower. First, we create the three buses as follows:

In [1]:
from pandapower.create import *
from pandapower.pf.runpp_3ph import runpp_3ph

In [2]:
net = create_empty_network()
b1  = create_bus(net, 20.0)
b2  = create_bus(net, 0.4)
b3  = create_bus(net, 0.4)

The external grid connection can then be created with the short-circuit parameters s_sc_max_mva and rx_max as well as the parameters x0x_max and r0x0_max that define the zero sequence impedances. The ext_grid parameters are all defined in the [ext_grid parameter documentation](https://pandapower.readthedocs.io/en/develop/elements/ext_grid.html#input-parameters).

In [3]:
create_ext_grid(net, b1, s_sc_max_mva=1000, rx_max=0.1, x0x_max=1.0,
                   r0x0_max=0.1)

np.int64(0)

The transformer requires the additional parameters vk0_percent, vkr0_percent, mag0_percent, mag0_rx, vector_group and si0_hv_partial for unbalanced calculation. The transformer parameters are all defined in the [transformer parameter documentation](https://pandapower.readthedocs.io/en/develop/elements/trafo.html#input-parameters).

In [4]:
create_transformer_from_parameters(net, b1, b2, sn_mva=0.63,
                                      vn_hv_kv=20., vn_lv_kv=0.4,
                                      vkr_percent=0.1, vk_percent=6,
                                      vk0_percent=6, vkr0_percent=0.78125,
                                      mag0_percent=100, mag0_rx=0.,
                                      pfe_kw=0.1, i0_percent=0.1,
                                      vector_group="Dyn", shift_degree=150,
                                      si0_hv_partial=0.9)

np.int64(0)

For lines, the zero sequence impedances r0_ohm_per_km, x0_ohm_per_km and c0_nf_per_km are required in addition to the balanced parameters. The line parameters are all defined in the [line parameter documentation](https://pandapower.readthedocs.io/en/develop/elements/line.html#input-parameters).

In [5]:
create_line_from_parameters(net, b2, b3, length_km=0.1, r0_ohm_per_km=0.0848,
                               x0_ohm_per_km=0.4649556, c0_nf_per_km=230.6,
                               max_i_ka=0.963, r_ohm_per_km=0.0212,
                               x_ohm_per_km=0.1162389, c_nf_per_km= 230)

np.int64(0)

Finally, to create an unbalanced power flow, we create an asymmetric load. The input parameters are all defined in the [asymmetric load parameter documentation](https://pandapower.readthedocs.io/en/develop/elements/asymmetric_load.html#input-parameters).

In [6]:
create_asymmetric_load(net, b3, p_a_mw=0.25, p_b_mw=0.18, p_c_mw=0.20, type="wye")

np.int64(0)

We can now run an unbalanced power flow:

In [7]:
runpp_3ph(net)

The results are stored in the result tables with suffix "_3ph", such as "res_bus_3ph", "res_line_3ph" etc.:

In [8]:
net.res_bus_3ph

,vm_a_pu,va_a_degree,vm_b_pu,va_b_degree,vm_c_pu,va_c_degree,p_a_mw,q_a_mvar,p_b_mw,q_b_mvar,p_c_mw,q_c_mvar,unbalance_percent
0,1.000025,-0.004229,0.999924,-119.999132,1.000051,120.003360,-0.233683,-0.014172,-0.2099,-0.047504,-0.192954,-0.010282,0.007796
1,0.990176,-154.176770,0.996375,87.028465,0.995592,-33.323592,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.686886
2,0.977551,-158.241097,1.001594,85.149362,0.974593,-35.449411,0.250000,0.000000,0.1800,0.000000,0.200000,0.000000,1.221488


We can check the voltage in all phases as well as the unbalance percentage according to IEC 62749:

In [9]:
net.res_bus_3ph[["vm_a_pu", "vm_b_pu", "vm_c_pu", "unbalance_percent"]]

,vm_a_pu,vm_b_pu,vm_c_pu,unbalance_percent
0,1.000025,0.999924,1.000051,0.007796
1,0.990176,0.996375,0.995592,0.686886
2,0.977551,1.001594,0.974593,1.221488


The line and trafo results provide currents and loadings in different phases or overall loading, which is defined as the maximum of all phase loadings.

In [10]:
net.res_line_3ph[["i_a_from_ka", "i_b_to_ka", "loading_b_percent", "loading_percent"]]

,i_a_from_ka,i_b_to_ka,loading_b_percent,loading_percent
0,1.107391,0.778182,80.808128,114.993847


In [11]:
net.res_trafo_3ph[["i_a_hv_ka", "i_b_lv_ka", "loading_b_percent", "loading_percent"]]

,i_a_hv_ka,i_b_lv_ka,loading_b_percent,loading_percent
0,0.020274,0.778182,102.487865,121.781392
